# Yfinance_Usecase 
- developing python definition funneled into langchain tool thus to be selected  

In [33]:
from typing import Type
from langchain.llms import OpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field

from datetime import datetime, timedelta

import yfinance as yf

In [34]:
def get_current_stock_price(ticker):
    """Method to get current stock price"""

    ticker_data = yf.Ticker(ticker)
    recent = ticker_data.history(period="6m")
    return {"price": recent.iloc[0]["Close"], \
            #"currency": ticker_data.info["currency"]
            }


def get_stock_performance(ticker, past_date):
    """Method to get stock price change in percentage"""
    past_date = datetime.strptime(past_date, '%Y-%m-%d')
    ticker_data = yf.Ticker(ticker)
    history = ticker_data.history(start=past_date)
    old_price = history.iloc[0]["Close"]
    current_price = history.iloc[-1]["Close"]
    return {"percent_change": ((current_price - old_price) / old_price) * 100}

class CurrentStockPriceInput(BaseModel):
    """Inputs for get_current_stock_price"""

    ticker: str = Field(description="Ticker symbol of the stock")


class CurrentStockPriceTool(BaseTool):
    name = "get_current_stock_price"
    description = """
        Useful when you want to get current stock price.
        You should enter the stock ticker symbol recognized by the yahoo finance
        """
    args_schema: Type[BaseModel] = CurrentStockPriceInput

    def _run(self, ticker: str):
        price_response = get_current_stock_price(ticker)
        return price_response

    def _arun(self, ticker: str):
        raise NotImplementedError("get_current_stock_price does not support async")



In [35]:
class StockPerformanceInput(BaseModel):
    """Inputs for get_stock_performance"""

    ticker: str = Field(description="Ticker symbol of the stock")
    past_date: str = Field(description="date in which stock price to be compared")


class StockPerformanceTool(BaseTool):
    name = "get_stock_performance"
    description = """
        Useful when you want to get the stock performance over the given period.
        You should enter the stock ticker symbol recognized by the yahoo finance
        """
    args_schema: Type[BaseModel] = StockPerformanceInput

    def _run(self, ticker: str , past_date: str):
        performance_response = get_stock_performance(ticker,past_date)
        return performance_response

    def _arun(self, ticker: str):
        raise NotImplementedError("get_stock_performance does not support async")

In [36]:
from langchain.agents import AgentType, initialize_agent, Tool
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.tools.tavily_search import TavilySearchResults
from langchain.utilities.tavily_search import TavilySearchAPIWrapper

In [37]:
from dotenv import load_dotenv
load_dotenv()

True

In [42]:
llm = ChatOpenAI(model="gpt-4", temperature=0, openai_api_key="")
search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(api_wrapper=search)
tools = [CurrentStockPriceTool(),StockPerformanceTool(),tavily_tool]

agent = initialize_agent(tools, llm, agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
agent.run("get me 3 data [\
          1. the latest trading day stock price of Ecopro(KOSDAQ), \
          2. the date in which Korea totally banned short-selling,\
          3. its performance since the ban]."
          "End search action till you find the date of ban. You'd better look up Korean journal first\
            For the failure of lookup after 3 rounds, Do not assume any random dates if the article searched doesn't say about it, you can tell me\
            the article's release date and ultimately the performance since then")



> Entering new AgentExecutor chain...
Thought: First, I need to find the latest trading day stock price of Ecopro(KOSDAQ). I will use the get_current_stock_price tool for this. 
Action:
```
{
  "action": "get_current_stock_price",
  "action_input": {
    "ticker": "086520.KQ"
  }
}
```
Observation: {'price': 667000.0}
Thought:The latest trading day stock price of Ecopro(KOSDAQ) is 667000.0. Now, I need to find the date in which Korea totally banned short-selling. I will use the tavily_search_results_json tool for this. I will search for "Korea short-selling ban date". 
Action:
```
{
  "action": "tavily_search_results_json",
  "action_input": {
    "query": "Korea short-selling ban date"
  }
}
```
Observation: [{'url': 'https://www.investopedia.com/how-does-south-koreas-short-selling-ban-affect-us-investors-8391133', 'content': 'Citing instances of illegal “naked” short selling, South Korea banned short selling through June 2024  How Does South Korea\'s Short Selling Ban Affect US Inv

'1. The latest trading day stock price of Ecopro(KOSDAQ) is 667000.0. 2. Korea totally banned short-selling starting from November 5, 2023. 3. The performance of Ecopro(KOSDAQ) since the ban has decreased by approximately 19.44%.'